## This is an example notebook to retrieve photometric catalog from IMAGE2 products by runnig photutils on _cal.fits files.

In [ ]:
import glob
import os
import argparse
from astropy.io import fits
#try:
from configobj import ConfigObj
#except ModuleNotFoundError:
#from jwst.extern.configobj.configobj import ConfigObj

from jwst.pipeline import collect_pipeline_cfgs
from jwst.pipeline import Detector1Pipeline
from jwst.pipeline import Image2Pipeline
from jwst.pipeline import Image3Pipeline
from jwst.pipeline import Spec2Pipeline
from jwst.pipeline import Spec3Pipeline

import yaml

In [ ]:
import jwst
jwst.__version__

In [ ]:
# Define some fancy function, if you want;
TERM = os.get_terminal_size()
SYM = "~"
def printnice(level):
    """
    Print the current pipeline level and make it nice :)
    Args:
        level (str): The thing you want to print nicely.
    """

    print("\n", SYM*TERM.columns, "\n", SYM, level.center(TERM.columns-2), SYM, "\n", SYM*TERM.columns, "\n", sep="")

In [ ]:
### Make a directory for reduced products;
DIR_RED = './reduced/'
if not os.path.exists(DIR_RED):
    os.mkdir(DIR_RED)


In [ ]:
import astropy.wcs as wcs
import numpy as np
import matplotlib.pyplot as plt
from photutils import Background2D, MedianBackground, detect_sources, deblend_sources, source_properties
from astropy.stats import gaussian_fwhm_to_sigma
from astropy.convolution import Gaussian2DKernel

images_cal = glob.glob('%s*_nis_cal.fits'%(DIR_RED))

for image_cal in images_cal[:]:

    hdu = fits.open(image_cal)
    data = hdu[1].data
    imwcs = wcs.WCS(hdu[1].header, hdu)
    err = hdu[2].data

    # Measure background and set detection threshold
    bkg_estimator = MedianBackground()
    bkg = Background2D(data, (50, 50), filter_size=(3, 3), bkg_estimator=bkg_estimator)
    #threshold = bkg.background + (15. * bkg.background_rms)
    threshold = bkg.background + (10. * bkg.background_rms)

    # Before detection, smooth image with Gaussian FWHM = n pixels
    sigma = 1.5 * gaussian_fwhm_to_sigma  

    kernel = Gaussian2DKernel(sigma)#, x_size=5, y_size=5)
    kernel.normalize()

    # Detect and deblend
    segm_detect = detect_sources(data, threshold, npixels=20)#, filter_kernel=kernel)

    # Save segmentation map of detected objects
    segm_hdu = fits.PrimaryHDU(segm_detect.data.astype(np.uint32), header=imwcs.to_header())
    segm_hdu.writeto(image_cal.replace('_cal.fits','_cal_seg.fits'), overwrite=True)
    
    # Save cat;
    cat = source_properties(data-bkg.background, segm_detect, wcs=imwcs, background=bkg.background, error=err)
    tbl = cat.to_table()#columns=columns)

    tbl.write(image_cal.replace('_cal.fits', '_cat_man.ecsv'), format='ascii.ecsv',overwrite=True)


